In [1]:
import os
import numpy as np
import scipy.sparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.tensor as T

import os
import pickle

from NQTM_torch import NQTM
from short_text_vae import Model
import config_911 as config
from components import *
%load_ext autoreload

In [25]:
#パラメータ
file_number = 1
device = torch.device(f'cuda:{file_number}')
config.learning_rate = 0.002
config.topic_num = 6
config.epoch_num = 60
batch_size = 64
K_d = 0.1
parent_dir = '/mnt/NAS0CAC8A/h-mukai'
fpath = '/mnt/NAS0CAC8A/h-mukai/NQTM/learning_'+str(file_number)+'.csv'#学習過程保存csv
#kl_min = 1.
#kl_t_n = 1600000
#kl_s_n = 480000
#tau_min = 0.5

In [19]:
PAD_TOKEN = '<pad>'  # バッチ処理の際に、短い系列の末尾を埋めるために使う （Padding）
BOS_TOKEN = '<s>'  # 系列の始まりを表す （Beggining of sentence）
EOS_TOKEN = '</s>'  # 系列の終わりを表す （End of sentence）
UNK_TOKEN = '<unk>'  # 語彙に存在しない単語を表す （Unknown）

MIN_COUNT = 5  # 語彙に含める単語の最低出現回数
PAD = 0
# 単語をIDに変換する辞書の初期値を設定
word2id = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: 1,
    EOS_TOKEN: 2,
    UNK_TOKEN: 3,
    }
# 単語辞書を作成(seqのtrain入力を使う)
vocab_X = Vocab(word2id=word2id)
vocab_X.build_vocab(X_train_seq, min_count=MIN_COUNT)#Validも含めたじしょ
vocab_size_X = len(vocab_X.id2word)
print('入力言語の語彙数：', vocab_size_X)

#IDへの変換
train_id = [sentence_to_ids(vocab_X, sentence) for sentence in train_data]
train_id_target = [sentence_to_ids(vocab_X, sentence) for sentence in train_data_t]
word2id = vocab_X.word2id
id2word = vocab_X.id2word

入力言語の語彙数： 5119


In [17]:
#stopwordsの含まれるor1文字以下のものは0カウント
#discriminator時にもこの処理が必要かな
from nltk.corpus import stopwords
sw =  stopwords.words('english')
sw += ['amp;','&amp;',"there's","this's","that's",'N%','N.','ht…','w','to…','f…','o…','yes','Nst',"i've","i'll",'th…','N/N','a…','like','new',"i'm",'N',"can't",'many',"he's",
          'could','need','would','people','one','still','may','might','<unk>','<pad>','<s>','</s>','N,N','.',"," "''","'s", ':', "'m", "'re"] 
#mask行列を作る
indices = []
for w,i in word2id.items():
    if w in sw or len(w)<=1:
        indices.append(word2id[w])
indices = T(indices)
mask = torch.sum(F.one_hot(indices, vocab_size_X),0)
mask = (torch.ones(vocab_size_X) - mask).to(torch.float32).to(device)

In [24]:
#データローダー
train_dataloader = DataLoader(train_id, train_id_target, batch_size, device, shuffle=True)

In [48]:
def kl_coef(step,n=20000):
    return min(1., step/n)
def print_top_words(beta, feature_names, n_top_words):
    top_words = list()
    for i in range(len(beta)):
        top_words.append(" ".join([feature_names[j] for j in beta[i].argsort()[:-n_top_words - 1:-1]]))
        print(top_words[-1])

def train(model, dataloader, config, optimizer,n):
    model.train()
    step = 0
    for epoch in range(config.epoch_num):
        w_omega = 0 if epoch < config.word_sample_epoch else 1.0
        train_loss = list()
        for batch in train_dataloader:
            inputs, targets, lengths = batch
            inputs = inputs.to(device);targets = targets.to(device);lengths = lengths.to(device)
            K_kl = kl_coef(step)
            tau = max(2 - step*(2/(len(X_train_seq)/16)/20),0.5)
            kl_s, mle_loss, loss_topic, loss_d = model(inputs, lengths, w_omega, targets, tau)
            loss = K_kl*kl_s+mle_loss+loss_topic+K_d*loss_d
            optimizer.zero_grad()
            loss.backward()#誤差伝搬(loss合計)
            optimizer.step()
            train_loss.append(loss.to('cpu').detach())
            step += 1
            if step%100 == 0:
                print_top_words(model.beta.weight.T.to('cpu').detach().numpy(), list(word2id.keys()),n)

        print('Epoch: ', '{:03d} loss: {:.3f}'.format(epoch + 1, np.mean(train_loss)))


In [60]:
%autoreload 2
from NQTM_torch import NQTM
from short_text_vae import Model
import config_911 as config
from components import *

In [61]:
model = Model(config, vocab_X, mask, device).to(device)

In [62]:
model = Model(config, vocab_X, mask, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
train(model, train_dataloader, config, optimizer,20)
#x=T(bow_matrix[0:2]).to(torch.float32).to(device)
#theta=model.encoder(x)
#theta

RuntimeError: The size of tensor a (256) must match the size of tensor b (5119) at non-singleton dimension 1

In [658]:
print_top_words(model.beta.weight.T.to('cpu').detach().numpy(), list(word2id.keys()),20)#replacementをFalse

views coulter exceeds wel hillary mayors mayor guess sheikh msg achievement mirecount alexander wore accessible concession thugs stopping martin acts
cuban democratic rejection serving utterly losses powerful crap chomsky lifting cuba products swamp doomed obamal class flint wealthy minori party
terrorist butter extends army champions advocates migrants doubles girl <unk> kill assaults gene arriving priests il porn uni terror injured
foreign vp kanye policy navarro allies blasts biafra queue selects retreat lou entanglements embassies peters kremlin uncharted shadow philippines cables
uncovers wi flash flight forces jason sandy baldwin ivanka nude seized dworkin privately published pushes alec joy articles megyn nasa
grandkids ratings exsists disbarred judges dni statistics tarmac reporter cia counselor amazing enthusiasm favourite becomes russia whisperer perfectly probe defecting


In [657]:
print_top_words(model.beta.weight.T.to('cpu').detach().numpy(), vocab)#replacementをTrue

TypeError: print_top_words() missing 1 required positional argument: 'n_top_words'

In [457]:
#path = './NQTM/output/my_research/theta_K6_1th.npy'
path = './NQTM/output/my_research/E_K6_1th.npy'
E_tf = np.load(path)

In [458]:
E_tf

array([[0.46060923, 0.10432205, 0.09942947, 0.10772838, 0.12851502,
        0.09787384],
       [0.1135954 , 0.44847798, 0.10744204, 0.11407465, 0.11449641,
        0.10668501],
       [0.1282088 , 0.11155075, 0.43263865, 0.1090094 , 0.10437302,
        0.11207238],
       [0.12951666, 0.12214497, 0.11223035, 0.41870385, 0.10767268,
        0.1107961 ],
       [0.13684861, 0.11960936, 0.09970406, 0.11118038, 0.44539905,
        0.08983887],
       [0.1407266 , 0.12159041, 0.10690624, 0.12019165, 0.11229575,
        0.4075454 ]], dtype=float32)

In [385]:
theta_tf[0]

array([0.14383471, 0.29047459, 0.16029118, 0.18976444, 0.0796748 ,
       0.13596028])

In [384]:
theta_tf[100]

array([0.04611887, 0.38681975, 0.03866256, 0.03293198, 0.02508899,
       0.47037786])

In [464]:
for i,batch in enumerate(train_dataloader):
    batch = batch.to(device)
    theta_torch = model.encoder(batch)
    if i==0:
        break

In [465]:
theta_torch.to('cpu').detach().numpy()[100]

array([0.3869409 , 0.08801629, 0.08779786, 0.087264  , 0.08757392,
       0.08765152, 0.08736151, 0.08739404], dtype=float32)

In [436]:
theta_torch.to('cpu').detach().numpy()[80]

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
      dtype=float32)

In [484]:
model.topic_dis_quant._E

Parameter containing:
tensor([[0.5311, 0.1068, 0.1309, 0.0782, 0.0873, 0.0779],
        [0.1155, 0.4706, 0.1320, 0.1044, 0.0705, 0.1063],
        [0.1079, 0.1046, 0.5217, 0.0949, 0.0814, 0.0857],
        [0.0849, 0.0981, 0.1661, 0.4318, 0.1278, 0.0865],
        [0.1134, 0.1129, 0.1079, 0.0883, 0.4558, 0.1231],
        [0.0969, 0.0916, 0.1204, 0.1241, 0.1058, 0.4635]], device='cuda:0',
       requires_grad=True)

In [459]:
model.topic_dis_quant._E.size()

torch.Size([8, 8])